## _Track Evaluation_

- Running _`trkx_from_gnn.py`_ produces recostucted tracks (track candiates)
- Running _`trkx_reco_eval.py`_ gives reconstruction evaluation: _`eff, fake rate, dup. rate, etc`_

Let's perform inspection after track evaluation.

In [ ]:
import sys, os, glob, yaml

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pprint
import seaborn as sns
import trackml.dataset

In [ ]:
import torch
from torch_geometric.data import Data
import itertools

In [ ]:
sys.path.append('..')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## _A. Inspection of Epsilon in DBSCAN_

1. Various $\epsilon$ for DBSCAN in `trkx_from_gnn.py`
2. Choose the best $\epsilon$ based on _`eff, fake rate, dup. rate`_) after `trkx_reco_eval.py`

**NOTE:** The $\epsilon$ in DBSCAN is alternative to chosing a cut on the `edge_score`.

In [ ]:
epsilon = [0.015, 0.025, 0.050, 0.075, 0.1, 0.15, 0.2, 0.25, 0.35, 0.45, 0.55, 0.75, 0.85, 0.95, 1.0]

In [ ]:
# Muons
eff_phys = [88.2250, 88.8337, 89.6347, 90.1322, 90.4533, 90.9524, 91.2334, 91.1832, 88.4393, 85.1644, 83.5178, 81.4443, 80.1778, 77.6377, 0.7349]
eff_tech = [89.8331, 90.4563, 91.2791, 91.7934, 92.1251, 92.6372, 92.9258, 92.8686, 90.0497, 86.7017, 85.0175, 82.9097, 81.6185, 79.0440, 0.7514]
purity   = [97.2793, 97.4626, 97.7950, 98.0192, 98.2108, 98.5698, 98.8359, 98.9482, 98.0389, 96.9250, 96.3454, 95.6658, 95.2468, 94.4137, 5.9280]
clones   = [14.9108, 14.2468, 13.3162, 12.7301, 12.2580, 11.4377, 10.4787, 9.0237, 4.5875, 2.3772, 1.7618, 1.3914, 1.3140, 1.2631, 0.1389]

In [ ]:
# FWP:
eff_phys = [87.2165, 87.3646, 87.4315, 87.4550, 87.3293, 87.0990, 86.8334, 86.5408, 85.4020, 84.0847, 82.9753, 80.6508, 78.4991, 73.2167, 16.5687]
eff_tech = [96.6127, 96.7740, 96.8500, 96.8766, 96.7327, 96.4860, 96.1941, 95.8648, 94.5731, 93.1120, 91.8883, 89.3128, 86.9306, 81.1078, 18.6763]
purity   = [99.3764, 99.4655, 99.5184, 99.5268, 99.5334, 99.5336, 99.5463, 99.5613, 99.6222, 99.4562, 99.2742, 98.2568, 98.7797, 96.8064, 47.7511]
clones   = [7.0048, 6.1859, 5.2207, 4.6082, 4.2689, 3.7827, 3.4842, 3.1144, 2.2536, 1.6902, 1.4752, 1.2884, 1.2231, 1.1924, 0.0000]

In [ ]:
# covert to ndarrays
eff_phys = np.array(eff_phys)
eff_tech = np.array(eff_tech)
purity   = np.array(purity)
clones   = np.array(clones)
epsilon = np.array(epsilon)

In [ ]:
# plt.style.use('ggplot')
# mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
# init subplots
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(8,6))
ax.plot(epsilon, purity, linestyle='-', linewidth=1.5, color='red', label="Tracking Purity")
ax.plot(epsilon, eff_phys, linestyle='-', linewidth=1.5, color='blue', label="Tracking Efficiency (Phys.)")
ax.plot(epsilon, eff_tech, linestyle='-', linewidth=1.5, color='orange', label="Tracking Efficiency (Tech.)")
ax.plot(epsilon, clones, linestyle='-', linewidth=1.5, color='green', label="Clone Rate")

plt.axvline(x=0.10, linewidth=0.5, color='k')
plt.axvline(x=0.15, linewidth=0.5, color='magenta')
plt.axvline(x=0.20, linewidth=0.5, color='k')
#plt.axvline(x=0.25, linewidth=0.5, color='magenta')
#plt.axvline(x=0.3, linewidth=0.5, color='magenta')

# plotting params
# ax.set_title('Track evaluation (efficiency vs epsilon)')
ax.set_xlabel('epsilon', fontsize=20)
#ax.set_ylabel('efficiency [%]', fontsize=20)
ax.set_xlim(0, 0.95)
# ax.set_ylim(0.0, 100)
ax.set_yscale('log')

start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 0.05))
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.2f'))


ax.grid(False)
ax.legend(fontsize=15, loc='best')
fig.tight_layout()
fig.savefig("fwp_epsilon_eff.pdf")

## _B. Visualize Test Data_

Inspect easy to difficult to reconstruct tracks. Choose 10 such events and use `score_cut=0.25`

In [ ]:
from src.drawing import detector_layout, draw_proc_event

In [ ]:
inputdir="../run_all/fwp_gnn_processed/pred"
proc_files = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
feature_data = torch.load(proc_files[0], map_location=device)

In [ ]:
feature_data

In [ ]:
event_id = int(os.path.basename(feature_data.event_file)[-4:])

In [ ]:
event_id

## _(C.) Post Analysis_

- Read evaluation data

In [ ]:
sys.path.append('..')

In [ ]:
from src.drawing import detector_layout, draw_proc_event
from src.utils_math import polar_to_cartesian

In [ ]:
# select event index (glob will fetch events as a list)
event_idx = 1

### _(1.) Read True Tracks_

In [ ]:
# get list of true tracks (test dataset)
inputdir="../run_all/fwp_gnn_processed/pred"
true_trkx = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
# event Data
true_track = torch.load(true_trkx[event_idx], map_location=device)
print("Length of Data: {}".format(len(true_track)))

In [ ]:
true_track

In [ ]:
# hit features
r, phi, ir = true_track.x.T

In [ ]:
# spatial coordinate transform
x, y = polar_to_cartesian(r.detach().numpy(), phi.detach().numpy())

In [ ]:
# compensate scaling
ir = ir.detach().numpy()*100

In [ ]:
# plot true event
fig, ax = detector_layout(figsize=(10,10))
e_id = int(true_track.event_file[-10:])
p_ids = np.unique(true_track.pid)

for pid in p_ids:
    idx = true_track.pid == pid
    ax.plot(x[idx], y[idx], '-', linewidth=1.5)
    ax.scatter(x[idx], y[idx], s=(ir[idx]*50), label='particle_id: {}'.format(int(pid)))

ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
# fig.savefig("true_track.png")

### _(2.) Read Reco Track_

In [ ]:
inputdir="../run_all/fwp_gnn_segmenting/seg"
reco_trkx = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
print("Track Id: ", reco_trkx[event_idx])

In [ ]:
reco_track = torch.load(reco_trkx[event_idx], map_location=device)
print("Length of Data: {}".format(len(reco_track)))

In [ ]:
reco_track.head()

In [ ]:
# plot reco event, here, (x,y,ir) comes from ture event. but idx is from reco event
fig, ax = detector_layout(figsize=(10,10))
e_id = e_id
t_ids = np.unique(reco_track.track_id)

for tid in t_ids:
    idx = reco_track.track_id == tid
    ax.plot(x[idx], y[idx], '-', linewidth=2)
    ax.scatter(x[idx], y[idx], s=(ir[idx]*100), label='particle_id: {}'.format(tid))
    
ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
# fig.savefig("reco_track.png")

### _(3.) Read Track Evaluation Data_

In [ ]:
inputdir="../run_all/fwp_gnn_segmenting/eval"

In [ ]:
evtid = int(os.path.split(reco_trkx[1006])[1])

In [ ]:
print("evtid: {}".format(evtid))

In [ ]:
!./eval_reco_evt.sh $evtid

In [ ]:
# path to HDF5 Store
hdf5_path = inputdir+"/{}_particles.h5".format(evtid)
print(hdf5_path)

In [ ]:
df = pd.read_hdf(hdf5_path)
print("Length of Data: {}".format(len(df)))

In [ ]:
df.head()

In [ ]:
# Read the Summary of Evaluation
sum_path = inputdir+"/{}_summary.txt".format(evtid)
print(sum_path)

In [ ]:
with open(sum_path) as f:
    contents = f.read()
    print(contents)